In [ ]:
import numpy as np
import pyvista as pv

from utils import load_nodes

In [ ]:
def to_polydata(fname, node_face, node_x, node_y, radius=None, start_index=True, panel=None, data=None, step=None, xy=None):
    if radius is None:
        radius = 1.0
        
    result = load_nodes(fname, node_face, node_x, node_y, radius, start_index=start_index, data=data, xy=xy)

    if data is not None:
        if xy:
            (x, y, z), faces, data, lons, lats = result
        else:
            (x, y, z), faces, data = result
    else:
        if xy:
            (x, y, z), faces, lons, lats = result
        else:
            (x, y, z), faces = result
        
    vertices = np.vstack((np.ravel(x), np.ravel(y), np.ravel(z))).T
    
    if panel is not None:
        N_panels = 6
        N_faces_per_panel = faces.shape[0] // N_panels
        s = slice(panel*N_faces_per_panel, (panel+1)*N_faces_per_panel)
        faces = faces[s]
        if data is not None and step is not None:
            data_points = data[step][s] 
    else:
        if data is not None and step is not None:
            data_points = data[step]
    
    pdata = pv.PolyData(vertices, faces)
    
    if data is not None and step is not None:
        pdata.cell_arrays["faces"] = data_points
    
    if xy:
        pdata.point_arrays["lons"] = lons
        pdata.point_arrays["lats"] = lats
    
    return pdata

In [ ]:
fname = "/home/bill/pyvista/data/test/real/qrclim.sst.ugrid.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"
data = "surface_temperature"
step = 0
panel = 0

In [ ]:
for s in range(12):
    pdata = to_polydata(fname, node_face, node_x, node_y, data=data, step=s, panel=None, xy=True)
    mesh_fname = f"pdata_xy_sst_t{s}.vtk"
    pdata.save(mesh_fname)
    for p in range(6):
        pdata = to_polydata(fname, node_face, node_x, node_y, data=data, step=s, panel=p, xy=True)
        mesh_fname = f"pdata_xy_sst_t{s}_panel{p}.vtk"
        pdata.save(mesh_fname)

In [ ]:
!ls -l pdata*.vtk

In [ ]:
pdata = to_polydata(fname, node_face, node_x, node_y, data=data, step=step, panel=None, xy=True)

In [ ]:
pdata

In [ ]:
p = pv.PlotterITK()
p.add_mesh(pdata)
#p.add_points(ugrid.points, color="green")
p.show()

In [ ]:
mesh = pv.read("pdata_sst_t0.vtk")

In [ ]:
mesh

In [ ]:
help(mesh.get_array)

In [ ]:
p = pv.PlotterITK()
p.add_mesh(mesh)
#p.add_points(mesh.points, color="red")
p.show()